In [19]:
import tensorflow as tf
import os
import keras
from keras.models import Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
import cv2
import numpy as np

In [20]:
os.chdir("../")

In [ ]:
model = tf.keras.models.load_model('model/model.h5')

In [ ]:
# Create a new model that excludes the last four layers
new_model = Model(inputs=model.input, outputs=model.layers[-5].output)

In [ ]:
for layer in new_model.layers:
    layer.trainable = False

In [ ]:
batch_size = 32
epochs = 25

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_gen = ImageDataGenerator(rescale=1./255)

train_generator = train_gen.flow_from_directory(
    'data',
    target_size=(200, 200), 
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale',
    shuffle=True
)
val_generator = validation_gen.flow_from_directory(
    'data',
    target_size=(200, 200),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale',
    shuffle=False
)

Found 200 images belonging to 4 classes.
Found 200 images belonging to 4 classes.


In [ ]:
base_output = new_model.output

x = Conv2D(64, (3, 3), activation='relu', padding='same')(base_output)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)

# Flatten after the convolutional layers
x = Flatten()(x)

# Add dense layers for classification or other purposes
x = Dense(220,activation = 'relu')(x)
x = Dense(128, activation='relu')(x)
output = Dense(4, activation='softmax')(x)

In [ ]:
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = new_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size
)